In [1]:
import pyterrier as pt
if not pt.started():
    pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
from pyterrier_colbert.ranking import ColBERTFactory
pytcolbert = ColBERTFactory("http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip", "./msmarco_index", "msmarco", gpu=True)

/home/stu1/s8/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to

[May 05, 11:32:04] #> Loading model checkpoint.
[May 05, 11:32:04] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


/home/stu1/s8/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[May 05, 11:32:38] #> checkpoint['epoch'] = 0
[May 05, 11:32:38] #> checkpoint['batch'] = 44500


In [4]:
msmarco_ds = pt.get_dataset("msmarco_passage")

# Get list 
qids = list(msmarco_ds.get_qrels("dev").loc[msmarco_ds.get_qrels("dev")['docno'].astype(int) < 100000]['qid'])

In [3]:
import numpy as np
from ir_measures import RR

results = []

for p in np.arange(0.0, 0.51, 0.05):
    token_ids = pytcolbert.top_p_tokens(p)
    dense_e2e = pytcolbert.end_to_end(token_ids)


    print(f"{p=}")
    results += [pt.Experiment(
        [dense_e2e],
        msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)],
        msmarco_ds.get_qrels("dev"),
        eval_metrics=["map", RR@10],
        names=[f"ColBERT ({p=})"]
    )]

    print(results[-1])

    del dense_e2e

[May 04, 20:18:29] #> Loading the FAISS index from ./msmarco_index/msmarco/ivfpq.faiss ..
[May 04, 20:18:29] #> Building the emb2pid mapping..
[May 04, 20:18:29] len(self.emb2pid) = 7409337
DEBUG>>> token_ids_to_prune={1012}
Loading reranking index, memtype=mem


Loading index shards to memory: 100%|██████████| 1/1 [00:00<00:00,  1.87shard/s]


DEBUG>>> vectors_ids_to_prune=array([     23,      54,      73, ..., 7409299, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 7024102
p=0.0
              name       map    RR@10
0  ColBERT (p=0.0)  0.480662  0.50672
DEBUG>>> token_ids_to_prune={1012}
DEBUG>>> vectors_ids_to_prune=array([     23,      54,      73, ..., 7409299, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 7024102
p=0.05
               name       map    RR@10
0  ColBERT (p=0.05)  0.480662  0.50672
DEBUG>>> token_ids_to_prune={1010, 1996, 1012}
DEBUG>>> vectors_ids_to_prune=array([      2,      13,      16, ..., 7409326, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 6425895
p=0.1
              name       map     RR@10
0  ColBERT (p=0.1)  0.480598  0.506185
DEBUG>>> token_ids_to_prune={1010, 1996, 1012, 1997}
DEBUG>>> vectors_ids_to_prune=array([      2,       4,      13, ..., 7409328, 7409331, 7409335])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove

In [6]:
for r in results:
    print(r)

              name       map    RR@10
0  ColBERT (p=0.0)  0.480662  0.50672
               name       map    RR@10
0  ColBERT (p=0.05)  0.480662  0.50672
              name       map     RR@10
0  ColBERT (p=0.1)  0.480598  0.506185
                              name       map     RR@10
0  ColBERT (p=0.15000000000000002)  0.480459  0.506023
              name     map     RR@10
0  ColBERT (p=0.2)  0.4805  0.506372
               name       map     RR@10
0  ColBERT (p=0.25)  0.487362  0.515169
                              name       map     RR@10
0  ColBERT (p=0.30000000000000004)  0.475224  0.501293
                              name       map     RR@10
0  ColBERT (p=0.35000000000000003)  0.472994  0.500411
              name       map     RR@10
0  ColBERT (p=0.4)  0.476822  0.503949
               name       map     RR@10
0  ColBERT (p=0.45)  0.476857  0.504794
              name       map     RR@10
0  ColBERT (p=0.5)  0.463068  0.492068


In [10]:
token_ids_p0 = pytcolbert.top_p_tokens(0)
dense_e2e_p0 = pytcolbert.end_to_end()

token_ids_p05 = pytcolbert.top_p_tokens(0.5)
dense_e2e_p05 = pytcolbert.end_to_end(token_ids_p05)

pt.Experiment(
    [dense_e2e_p0, dense_e2e_p05],
    msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)],
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map", RR@10],
    baseline=0,
    names=["p=0.0", "p=0.5"]
)

DEBUG>>> token_ids_to_prune={1024, 1025, 2, 2051, 2036, 2049, 2054, 2055, 2048, 2057, 2037, 2058, 2060, 1037, 2062, 2061, 2064, 2065, 2063, 2066, 1020, 1045, 2070, 2069, 2040, 2075, 2041, 1055, 1056, 2079, 2042, 2084, 2086, 2043, 2087, 2089, 2090, 2044, 2095, 2097, 2045, 2046, 2102, 1029, 2047, 2107, 2109, 2110, 2111, 2115, 2116, 2121, 2122, 2050, 2128, 2129, 2131, 1035, 2053, 101, 102, 2151, 2154, 2156, 2566, 2163, 2059, 2171, 2184, 2191, 2193, 2199, 2224, 2256, 2300, 2026, 1523, 1003, 1524, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 1016, 2011, 2012, 2013, 2010, 2015, 2017, 2019, 2021, 2022, 2023, 2024, 2025, 1002, 2027, 2028, 2029, 1005, 1006, 2030, 1007, 2034, 2031, 1010, 1011, 1012, 2039, 1521, 1013, 1018, 1019, 1015, 1017, 2038, 2035}
DEBUG>>> vectors_ids_to_prune=array([      0,       1,       2, ..., 7409333, 7409335, 7409336])
DEBUG>>> pre-remove: 7409337
DEBUG>>> post-remove: 3701713


,name,map,RR@10,map +,map -,map p-value,RR@10 +,RR@10 -,RR@10 p-value
0,p=0.0,0.481221,0.507304,NaN,NaN,NaN,NaN,NaN,NaN
1,p=0.5,0.463068,0.492068,50.0,54.0,0.110796,40.0,41.0,0.202347


In [5]:
index_ref = pt.IndexRef.of("./msmarco_index/msmarco")
index = pt.IndexFactory.of(index_ref)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pt.Experiment(
    [bm25],
    msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)],
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map", RR@10],
    names=["BM25"]
)

JavaException: JVM exception occurred: No IndexLoaders were supported for indexref ./msmarco_index/msmarco; It may be your ref has the wrong location. Alternatively, Terrier is misconfigured - did you import the correct package to deal with this indexref? java.lang.UnsupportedOperationException

In [ ]:
token_ids = pytcolbert.top_p_tokens(0.3)
dense_e2e = pytcolbert.end_to_end(token_ids)

In [ ]:
msmarco_ds = pt.get_dataset("msmarco_passage")

# Get list 
qids = list(msmarco_ds.get_qrels("dev").loc[msmarco_ds.get_qrels("dev")['docno'].astype(int) < 100000]['qid'])

pt.Experiment(
    [dense_e2e],
    msmarco_ds.get_topics("dev").loc[msmarco_ds.get_topics("dev")['qid'].str.contains('|'.join(qids), na=False)].head(10),
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map. mrr@10"],
    names=["ColBERT"]
)


In [ ]:
result = dense_e2e.search("cost of endless pools swim spa")

In [ ]:
dense_e2e.search("what is pcnt")